In [1]:
import os, glob, cv2, dlib
import numpy as np
from imutils import face_utils

from sklearn.model_selection import train_test_split

from progressbar import ProgressBar

In [2]:
data_dir = 'dataset/KomNet/original_sosmed/'
output_dir = 'dataset/KomNet_masked2/original_sosmed/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [3]:
### Proses Listing Dataset
print('\n##### PROSES LISTING DATASET #####')
print('-------------------------------------')

labels = os.listdir(data_dir)
labels.sort()

X = list()
y = list()

for label in labels:
    dir=os.path.join(data_dir,label)
    n = 0
    for file in glob.glob(dir + "/*"):
        X.append(file)
        y.append(label)
        n+=1
    print('Listing data pada label',label,'telah selesai. Jumlah data :', str(n))
    
X = np.asarray(X)
y = np.asarray(y)


##### PROSES LISTING DATASET #####
-------------------------------------
Listing data pada label 10_sosmed_i made parasya maharta telah selesai. Jumlah data : 24
Listing data pada label 11_sosmed_norasela telah selesai. Jumlah data : 24
Listing data pada label 12_sosmed_gede agus andika pratama telah selesai. Jumlah data : 24
Listing data pada label 13_sosmed_narendra brillian telah selesai. Jumlah data : 24
Listing data pada label 14_sosmed_gusti ayu nyoman sita wahana murti telah selesai. Jumlah data : 24
Listing data pada label 15_sosmed_richard putra rosul telah selesai. Jumlah data : 24
Listing data pada label 16_sosmed_ni kadek sitayanti telah selesai. Jumlah data : 24
Listing data pada label 17_sosmed_i komang agus merta dana telah selesai. Jumlah data : 24
Listing data pada label 18_sosmed_i kadek yogi ardana telah selesai. Jumlah data : 24
Listing data pada label 19_sosmed_i gede satya bramananda telah selesai. Jumlah data : 24
Listing data pada label 1_sosmed_i made dwiki sa

In [4]:
# Train test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(958,) (958,) (240,) (240,)


In [5]:
# Fungsi deteksi 68 facial points dengan DLIB
# ============================================

def facial_points(image):
    gray = image
    if(len(image.shape)==3):
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    gray = cv2.equalizeHist(gray)
    
    detector = dlib.get_frontal_face_detector() 
    predictor = dlib.shape_predictor("model/shape_predictor_68_face_landmarks.dat") 
    rects = detector(gray, 1) 
    
    shape = np.array([0])
    
    for (i, rect) in enumerate(rects):  
        shape = predictor(gray, rect) 
        shape = face_utils.shape_to_np(shape) 

    return shape

In [6]:
def masked_face(image):
    shape = facial_points(image)
    
    # Buat citra wajah dengan facial points
    img_fp = image.copy()
    for p in shape:
        img_fp = cv2.circle(img_fp, p, 5, (0, 255, 0) , -1)
    
    # Buat list facial points untuk augmentasi masker
    pts = []
    for i in range(15):
        pts.append(shape[i+1])
    pts.append(shape[28])
    
    pts = np.array(pts)
    pts = pts.reshape((-1, 1, 2))  

    image = cv2.fillPoly(image, [pts], (255, 255, 255))
    
    return image, img_fp

In [7]:
from datetime import datetime

logfile = output_dir + 'masked_log.txt'

if not os.path.exists(logfile):
    f = open(logfile, "w")
    f.write(str(datetime.now()) + '\n' + '===========================\n\n')
    f.close()

In [8]:
pbar = ProgressBar()

test_dir = output_dir + 'test/'
n=0
for file in pbar(X_test):
    label = y_test[n]
    d = 0
    
    d0, d1 = label.rfind('_'), label.rfind('-')

    if d0>d1:
        d = d0+1
    else:
        d = d1+1
    
    label = label[d:]
    label = label.lower()
    label = label.strip()
    
    if not os.path.exists(test_dir + label):
        os.makedirs(test_dir + label)
    filename = os.path.basename(file)
    output = test_dir + label + '/' + filename + '.jpg'
    img = cv2.imread(file)
    try:
        img = masked_face(img)[0]
        cv2.imwrite(output, img)
        log = 'Done' + '|' + label + '|' + filename + '|' + output + '\n'
    except:
        log = 'Error' + '|' + label + '|' + filename + '|' + output + '\n'

    f = open(logfile, "a")
    f.write(log)
    f.close()
    n+=1

100% |########################################################################|


In [9]:
pbar = ProgressBar()

train_dir = output_dir + 'train/'
n=0
for file in pbar(X_train):
    label = y_train[n]
    d = 0
    
    d0, d1 = label.rfind('_'), label.rfind('-')

    if d0>d1:
        d = d0+1
    else:
        d = d1+1
    
    label = label[d:]
    label = label.lower()
    label = label.strip()
    
    if not os.path.exists(train_dir + label):
        os.makedirs(train_dir + label)
    filename = os.path.basename(file)
    output = train_dir + label + '/' + filename + '.jpg'
    img = cv2.imread(file)
    cv2.imwrite(output, img)

    f = open(logfile, "a")
    f.write(log)
    f.close()
    n+=1

100% |########################################################################|
